In [263]:

import requests # to make TMDB API calls
import json
from config import api_key #in order for personal API to not be exposed
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


In [265]:
# Import sqlite3, then create the SQL database/connect to it (creating the database connection will create a DB)
import sqlite3

conn = sqlite3.connect('movies_table1.db') # connecting to new created data base

c = conn.cursor() #Used c vs. cursor for shorthand


In [266]:
#writing the basic function for creating the columns of our sqlite data base
create_query = """CREATE TABLE IF NOT EXISTS movies_table1
        (id INTEGER ,title TEXT, popularity INTEGER, release_date TEXT,vote_average REAL, vote_count INTEGER)"""

def create_table(query):
    c.execute(query)
    
create_table(create_query)

In [267]:
#creating the basic function in otrder to insert data to our sqlite data base
def insert_query(query):
    
        for j in range(len(query)): 
            c.execute("INSERT INTO movies_table1 (id,title,popularity,release_date,vote_average,vote_count) VALUES (?,?,?,?,?,?)",
                  (query[j]['id'],query[j]['title'],query[j]['popularity'],query[j]['release_date'],query[j]['vote_average'],query[j]['vote_count']))


In [297]:
#creating the loop that approacing the API of the movie data base

def creating_list_for_db(first_page,last_page):
    
    #make sure the input is correct. the API can give us 20 results in each round and we know the data base has 353 pages
    if type(first_page) != int or first_page <1 or first_page>354:
        return "Error in first page input"
    elif type(last_page) != int or last_page <2 or last_page>354:
        return "Error in last page input"
    else:
    
        try:
            counter=0 #we created a counter, that in case the loop is broken we know in which point to continue
            for i in range(first_page,last_page+1):
                response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key='+ api_key+'&language=en-US&page='+str(i))
                
                if response.status_code != 200: 
                    print(response.status_code) #if there was a problem with the connection to the API we would like to know the reason
                    print (counter) 
                    
                else:
                data= response.json()
                results = data['results'] # we are interest only in the result part from the data base
                insert_query(results)  # we are inserting the categories from result we would like to have into our sqlite data base
                counter +=1
                time.sleep(5)
            
            
        except (RuntimeError, TypeError, NameError,ValueError): #in case of error we would like to loop to not loose all the information
             pass
    
    return "Data base is ready!"
    

sqldb = c.execute("""select * from movies_table1;""").fetchall() #we are fetcing the infomation from the data base we created





In [307]:
#we are converting our sqlite data base to pandas data frame to have better outlook of our data
df=pd.DataFrame(sqldb)
df.columns=['ID', 'Title', 'Popularity','Release_Date','Vote_Average','Vote_Count'] # adding titles to our Data Frame


In [172]:
#we are exporting our data to csv for our future use
df.to_csv("data_base_movies.csv")


In [50]:
c.close() # closing the connection
conn.close()